In [1]:
# Import all the necessary modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas_datareader as pdr
import math
import datetime
import itertools
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from trend_following_signal import (apply_jupyter_fullscreen_css, load_financial_data, get_returns_volatility, calculate_slope, trend_signal, slope_signal, 
                             create_trend_strategy, get_close_prices, calculate_donchian_channels)
from strategy_performance import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
import coinbase_utils as cn
import strategy_performance as perf
import position_sizing as size
import trend_following_signal as tf
%matplotlib inline

In [2]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)
importlib.reload(size)

<module 'position_sizing' from '/Users/adheerchauhan/Documents/git/trend_following/position_sizing.py'>

In [3]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

In [7]:
client = cn.get_coinbase_rest_api_client(cn.key_location)

In [7]:
def get_coinbase_daily_historical_price_data_hour(client, ticker, start_timestamp, end_timestamp, retries=3, delay=5, granularity='ONE_HOUR'):
    # granularity = 'ONE_DAY'  # Daily granularity
    attempts = 0

    while attempts < retries:
        try:
            # Attempt to fetch the candles
            candle_list = client.get_candles(
                product_id=ticker,
                start=start_timestamp,
                end=end_timestamp,
                granularity=granularity
            ).candles

            # Process candle data
            candle_data = []
            for candles in candle_list:
                candle_info = {
                    'date': candles['start'],
                    'low': float(candles['low']),
                    'high': float(candles['high']),
                    'open': float(candles['open']),
                    'close': float(candles['close']),
                    'volume': float(candles['volume'])
                }
                candle_data.append(candle_info)

            # Convert to DataFrame
            df_candles = pd.DataFrame(candle_data).sort_values('date')
            df_candles['date'] = pd.to_datetime(df_candles['date'], unit='s').dt.date
            df_candles = df_candles.set_index('date')
            # df_candles['ticker'] = ticker

            return df_candles, candle_list

        except requests.exceptions.ConnectionError as e:
            print(f"Connection error: {e}. Retrying in {delay} seconds...")
            attempts += 1
            time.sleep(delay)

    # If all retries fail, raise the error
    raise Exception("Max retries exceeded. Could not connect to Coinbase API.")

In [9]:
import requests
import time
import datetime
import pandas as pd

def get_coinbase_price_at_timestamp(client, ticker, start_date, end_date, timestamp='08:00:00', retries=3, delay=5):
    """
    Fetches the price of a cryptocurrency at a specified timestamp for using the Coinbase REST API.

    Parameters:
    - client: The Coinbase API client
    - ticker: The crypto trading pair (e.g., 'BTC-USD')
    - retries: Number of retry attempts in case of failure
    - delay: Delay between retries in seconds

    Returns:
    - A Pandas DataFrame with date and 8 AM price
    """

    date_list = pd.date_range(start=start_date, end=end_date, freq="D")
    price_data = []

    for date in date_list:
        start_timestamp = int(datetime.datetime.strptime(f'{date.date()} {timestamp}','%Y-%m-%d %H:%M:%S').timestamp())
        end_timestamp = int(datetime.datetime.strptime(f'{date.date()} {timestamp}','%Y-%m-%d %H:%M:%S').timestamp())

        attempts = 0
        while attempts < retries:
            try:
                # Fetch 8 AM hourly candle
                candle_list = client.get_candles(
                    product_id=ticker,
                    start=start_timestamp,
                    end=end_timestamp,
                    granularity='ONE_MINUTE'  # Minute candles
                ).candles

                if candle_list:
                    # Extract price at timestamp
                    candles = candle_list[0]  # [timestamp, low, high, open, close, volume]
                    price_data.append({
                        'date': datetime.datetime.utcfromtimestamp(int(candles['start'])).date(),
                        '8am_open_price': float(candles['open']),
                        '8am_close_price': float(candles['close'])
                    })

                break  # Exit retry loop if successful

            except requests.exceptions.ConnectionError as e:
                print(f"Connection error: {e}. Retrying in {delay} seconds...")
                attempts += 1
                time.sleep(delay)

        if attempts == retries:
            print(f"Failed to fetch data for {date.strftime('%Y-%m-%d')} after {retries} retries.")

    # Convert to DataFrame
    df_prices = pd.DataFrame(price_data)
    df_prices.set_index('date', inplace=True)

    return df_prices

In [11]:
def save_historical_crypto_prices_at_timestamp_from_coinbase(ticker_list, user_start_date=False, start_date=None, end_date=None, window_days=30, timestamp='08:00:00',
                                                             save_to_file=False):

    client = cn.get_coinbase_rest_api_client(cn.key_location)
    if user_start_date:
        start_date = pd.Timestamp(start_date).date()
    else:
        start_date = cn.coinbase_start_date_by_ticker_dict.get(ticker)
        start_date = pd.Timestamp(start_date).date()
        if not start_date:
            print(f"Start date for {ticker} is not included in the dictionary!")
            return None

    if pd.Timestamp(end_date).date() > datetime.datetime.now().date():
        end_date = datetime.datetime.now().date()
    
    price_dict = {}
    for ticker in ticker_list:
        current_start_date = start_date
        current_end_date = current_start_date + datetime.timedelta(days=window_days)
        cols = [f'{ticker}_8am_open_price', f'{ticker}_8am_close_price']
        df = pd.DataFrame(columns=cols)
        while current_start_date < pd.Timestamp(end_date).date():
            if (current_start_date < pd.Timestamp(end_date).date()) & (current_end_date > pd.Timestamp(end_date).date()):
                current_end_date = pd.Timestamp(end_date).date()
            print(ticker, current_start_date, current_end_date)
            _df = get_coinbase_price_at_timestamp(client, ticker, current_start_date,
                                                  current_end_date, timestamp)
            _df.columns = cols
            df = pd.concat([df, _df])
            current_start_date = current_end_date + datetime.timedelta(days=1)
            current_end_date = current_end_date + datetime.timedelta(days=window_days)
    
        if save_to_file:
            filename = f"{ticker}-{timestamp}_pickle-{start_date.strftime('%Y-%m-%d')}-{end_date.strftime('%Y-%m-%d')}"
            output_file = f'coinbase_historical_price_folder/{filename}'
            df.to_pickle(output_file)

        price_dict[ticker] = df
    
    df_price = pd.concat(price_dict, axis=1)
    df_price.columns = df_price.columns.droplevel(level=0)

    return df_price

In [473]:
start_date = '2021-06-01'
end_date = '2025-03-31'
df_prices = save_historical_crypto_prices_at_timestamp_from_coinbase(ticker_list, user_start_date=True, start_date=start_date, end_date=end_date, save_to_file=False)

In [457]:
df_prices.tail(200)

,BTC-USD_8am_open_price,BTC-USD_8am_close_price,ETH-USD_8am_open_price,ETH-USD_8am_close_price,LTC-USD_8am_open_price,LTC-USD_8am_close_price,SOL-USD_8am_open_price,SOL-USD_8am_close_price
2024-08-12,59874.43,59797.06,2674.47,2671.77,61.70,61.67,150.33,150.18
2024-08-13,58845.51,58849.89,2645.12,2645.93,61.79,61.76,145.91,146.13
2024-08-14,61340.31,61327.00,2753.05,2752.12,63.67,63.64,148.09,148.06
2024-08-15,58744.40,58777.77,2637.68,2638.90,65.28,65.32,143.12,143.22
2024-08-16,58412.61,58421.66,2612.74,2614.65,65.72,65.69,142.02,142.13
2024-08-17,59135.31,59127.07,2599.63,2598.67,67.31,67.35,139.14,139.03
2024-08-18,60020.79,59991.77,2650.05,2648.70,67.62,67.61,144.62,144.54
2024-08-19,58404.81,58455.94,2588.23,2588.04,66.83,66.91,142.37,142.42
2024-08-20,60547.22,60541.69,2643.78,2643.55,66.16,66.17,145.43,145.42
2024-08-21,59374.84,59391.77,2577.91,2577.77,62.77,62.77,141.32,141.28


In [469]:
end_date

'2025-03-31'

In [477]:
start_date = '2021-06-01'
end_date = '2025-02-27'
ticker = 'BTC-USD'
filename = f"{ticker}-08:00:00_pickle-{pd.to_datetime(start_date).date().strftime('%Y-%m-%d')}-{pd.to_datetime(end_date).date().strftime('%Y-%m-%d')}"
df_test = pd.read_pickle(f'coinbase_historical_price_folder/{filename}')

In [481]:
df_test.head(200)

,BTC-USD_8am_open_price,BTC-USD_8am_close_price
2021-06-01,36601.20,36545.54
2021-06-02,37204.26,37242.00
2021-06-03,38791.36,38811.27
2021-06-04,36587.77,36403.02
2021-06-05,35915.26,35821.11
2021-06-06,35893.44,35858.47
2021-06-07,36395.00,36424.28
2021-06-08,33123.94,33153.76
2021-06-09,35017.72,35199.99
2021-06-10,37843.96,37787.97


In [66]:
def load_coinbase_timestamp_price_pickle_file(ticker):
    start_date = pd.Timestamp('2021-06-01').date()
    end_date = pd.Timestamp('2025-02-27').date()
    filename = f"{ticker}-8_am_pickle-{start_date.strftime('%Y-%m-%d')}-{end_date.strftime('%Y-%m-%d')}"
    df = pd.read_pickle(f'coinbase_historical_price_folder/{filename}')

    return df

In [15]:
def create_trend_strategy_timestamp(ticker_list, mavg_start=50, mavg_end=250, mavg_stepsize=6):
    ticker_dict = {}
    for ticker in ticker_list:
        df = load_coinbase_timestamp_price_pickle_file(ticker)
        price_col = f'{ticker}_8am_open_price'
        df[f'{ticker}_pct_returns'] = df[price_col].pct_change()
    
        for window in np.linspace(mavg_start, mavg_end, mavg_stepsize):
            df[f'{ticker}_{int(window)}_mavg'] = df[f'{price_col}'].ewm(span=window).mean()
            
        ## Ticker Trend Signal and Trade
        mavg_col_list = [f'{ticker}_{int(mavg)}_mavg' for mavg in np.linspace(mavg_start, mavg_end, mavg_stepsize).tolist()]
        df[f'{ticker}_trend_signal'] = df[mavg_col_list].apply(tf.trend_signal, axis=1).shift(1)

        ticker_dict[ticker] = df

    df_final = pd.concat(ticker_dict, axis=1)
    df_final.columns = df_final.columns.droplevel(level=0)

    return df_final

In [19]:
ticker_list = ['BTC-USD','ETH-USD','LTC-USD','SOL-USD']#,'XRP-USD']

In [21]:
df_trend = create_trend_strategy_timestamp(ticker_list)

In [38]:
def calculate_donchian_channels_timestamp(ticker_list):
    start_date = '2021-06-01'
    end_date = '2025-02-27'
    rolling_donchian_window = 20
    ticker_dict = {}
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()
    for ticker in ticker_list:
        df = load_coinbase_timestamp_price_pickle_file(ticker)
        price_col = f'{ticker}_8am_open_price'
        
        # Rolling maximum of returns (upper channel)
        df[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price'] = (
            df[price_col].rolling(window=rolling_donchian_window).max())
    
        # Rolling minimum of returns (lower channel)
        df[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price'] = (
            df[price_col].rolling(window=rolling_donchian_window).min())
    
        # Middle of the channel (optional, could be just average of upper and lower)
        df[f'{ticker}_{rolling_donchian_window}_donchian_middle_band_price'] = (
                (df[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price'] +
                 df[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price']) / 2)
    
        # Shift only the Keltner channel metrics to avoid look-ahead bias
        df[[f'{ticker}_{rolling_donchian_window}_donchian_middle_band_price',
            f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price',
            f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price']] = df[[
            f'{ticker}_{rolling_donchian_window}_donchian_middle_band_price',
            f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price',
            f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price']].shift(1)

        # Donchian Buy signal: Price crosses above upper band
        # Donchian Sell signal: Price crosses below lower band
        donchian_signal_col = f'{ticker}_{rolling_donchian_window}_donchian_signal'
        df[donchian_signal_col] = np.where(
            (df[price_col] > df[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price']), 1,
            np.where((df[price_col] < df[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price']),
                     -1, 0))

        ticker_dict[ticker] = df

    df_final = pd.concat(ticker_dict, axis=1)
    df_final.columns = df_final.columns.droplevel(level=0)

    return df_final

In [40]:
df_donchian = calculate_donchian_channels_timestamp(ticker_list)

In [42]:
def get_portfolio_trend_donchian_signal(ticker_list, fast_mavg=50, mavg_stepsize=6, slow_mavg=250, rolling_donchian_window=20, long_only=True):

    df_trend = create_trend_strategy_timestamp(ticker_list)
    df_donchian = calculate_donchian_channels_timestamp(ticker_list)

    donchian_cols = []
    for ticker in ticker_list:
        cols = [f'{ticker}_{rolling_donchian_window}_donchian_upper_band_price', f'{ticker}_{rolling_donchian_window}_donchian_lower_band_price',
                f'{ticker}_{rolling_donchian_window}_donchian_middle_band_price', f'{ticker}_{rolling_donchian_window}_donchian_signal']
        donchian_cols.extend(cols)
    df_final = pd.merge(df_trend, df_donchian[donchian_cols], left_index=True, right_index=True, how='left')
    
    for ticker in ticker_list:
        trend_donchian_signal_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'
        donchian_signal_col = f'{ticker}_{rolling_donchian_window}_donchian_signal'
        trend_signal_col = f'{ticker}_trend_signal'
        
        # Trend and Donchian Channel Signal
        buy_signal = ((df_final[donchian_signal_col] == 1) &
                      (df_final[trend_signal_col] == 1))
        sell_signal = ((df_final[donchian_signal_col] == -1) &
                       (df_final[trend_signal_col] == -1))
        
        # Generate Long Only Signal
        if long_only:
            df_final[trend_donchian_signal_col] = np.where(buy_signal, 1, 0)
        # Generate Long & Short Signal
        else:
            df_final[trend_donchian_signal_col] = np.where(buy_signal, 1, np.where(sell_signal, -1, 0))

    return df_final

In [44]:
df_signal = get_portfolio_trend_donchian_signal(ticker_list)

In [46]:
print(df_signal.shape)

(1368, 60)


In [48]:
df_signal.head()

,BTC-USD_8am_open_price,BTC-USD_8am_close_price,BTC-USD_pct_returns,BTC-USD_50_mavg,BTC-USD_90_mavg,BTC-USD_130_mavg,BTC-USD_170_mavg,BTC-USD_210_mavg,BTC-USD_250_mavg,BTC-USD_trend_signal,ETH-USD_8am_open_price,ETH-USD_8am_close_price,ETH-USD_pct_returns,ETH-USD_50_mavg,ETH-USD_90_mavg,ETH-USD_130_mavg,ETH-USD_170_mavg,ETH-USD_210_mavg,ETH-USD_250_mavg,ETH-USD_trend_signal,LTC-USD_8am_open_price,LTC-USD_8am_close_price,LTC-USD_pct_returns,LTC-USD_50_mavg,LTC-USD_90_mavg,LTC-USD_130_mavg,LTC-USD_170_mavg,LTC-USD_210_mavg,LTC-USD_250_mavg,LTC-USD_trend_signal,SOL-USD_8am_open_price,SOL-USD_8am_close_price,SOL-USD_pct_returns,SOL-USD_50_mavg,SOL-USD_90_mavg,SOL-USD_130_mavg,SOL-USD_170_mavg,SOL-USD_210_mavg,SOL-USD_250_mavg,SOL-USD_trend_signal,BTC-USD_20_donchian_upper_band_price,BTC-USD_20_donchian_lower_band_price,BTC-USD_20_donchian_middle_band_price,BTC-USD_20_donchian_signal,ETH-USD_20_donchian_upper_band_price,ETH-USD_20_donchian_lower_band_price,ETH-USD_20_donchian_middle_band_price,ETH-USD_20_donchian_signal,LTC-USD_20_donchian_upper_band_price,LTC-USD_20_donchian_lower_band_price,LTC-USD_20_donchian_middle_band_price,LTC-USD_20_donchian_signal,SOL-USD_20_donchian_upper_band_price,SOL-USD_20_donchian_lower_band_price,SOL-USD_20_donchian_middle_band_price,SOL-USD_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_signal,ETH-USD_50_6_250_mavg_crossover_20_donchian_signal,LTC-USD_50_6_250_mavg_crossover_20_donchian_signal,SOL-USD_50_6_250_mavg_crossover_20_donchian_signal
2021-06-01,36601.20,36545.54,NaN,36601.200000,36601.200000,36601.200000,36601.200000,36601.200000,36601.200000,NaN,2624.78,2616.32,NaN,2624.780000,2624.780000,2624.780000,2624.780000,2624.780000,2624.780000,NaN,181.80,181.47,NaN,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0
2021-06-02,37204.26,37242.00,0.016477,36908.760600,36906.080333,36905.049462,36904.503706,36904.165857,36903.936120,-1.0,2695.44,2699.21,0.026920,2660.816600,2660.502556,2660.381769,2660.317824,2660.278238,2660.251320,-1.0,182.47,182.76,0.003685,182.141700,182.138722,182.137577,182.136971,182.136595,182.136340,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0
2021-06-03,38791.36,38811.27,0.042659,37561.559032,37548.523069,37543.517641,37540.869664,37539.231128,37538.117228,1.0,2808.70,2810.43,0.042019,2712.095726,2711.003483,2710.583721,2710.361574,2710.224083,2710.130602,1.0,192.28,192.21,0.053762,185.657194,185.594544,185.570529,185.557835,185.549982,185.544646,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0
2021-06-04,36587.77,36403.02,-0.056806,37303.314152,37300.269887,37299.038744,37298.373504,37297.957035,37297.671807,1.0,2625.76,2607.40,-0.065133,2689.199844,2688.977047,2688.885933,2688.836483,2688.805452,2688.784168,1.0,173.85,172.76,-0.095850,182.525974,182.559820,182.572435,182.579022,182.583069,182.585807,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0
2021-06-05,35915.26,35821.11,-0.018381,37003.059475,37010.821469,37013.702441,37015.203975,37016.125248,37016.748104,1.0,2642.08,2633.10,0.006215,2679.007191,2679.176196,2679.234512,2679.263870,2679.281516,2679.293285,1.0,173.75,173.04,-0.000575,180.627614,180.718686,180.753241,180.771427,180.782646,180.790258,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0


In [56]:
def get_returns_volatility_timestamp(df, vol_range_list=[10], close_px_col='BTC-USD'):
    df[f'{close_px_col}_pct_returns'] = df[f'{close_px_col}_8am_open_price'].pct_change()
    for vol_range in vol_range_list:
        df[f'{close_px_col}_volatility_{vol_range}'] = df[f'{close_px_col}_pct_returns'].rolling(vol_range).std()

    return df

In [60]:
def get_volatility_adjusted_trend_signal_timestamp(df, ticker_list, volatility_window=20, fast_mavg=50, mavg_stepsize=6, slow_mavg=250,
                                         rolling_donchian_window=20, annual_trading_days=365):
    ticker_signal_dict = {}
    final_cols = []
    for ticker in ticker_list:
        trend_signal_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'
        trend_returns_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'
        trend_trades_col = f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_trades'
        annualized_volatility_col = f'{ticker}_annualized_volatility_{volatility_window}'
        vol_adj_trend_signal_col = f'{ticker}_vol_adjusted_trend_signal'

        ## Calculate Position Volatility Adjusted Trend Signal
        df = get_returns_volatility_timestamp(df, vol_range_list=[volatility_window], close_px_col=f'{ticker}')
        df[annualized_volatility_col] = (df[f'{ticker}_volatility_{volatility_window}'] *
                                         np.sqrt(annual_trading_days))
        df[vol_adj_trend_signal_col] = (df[trend_signal_col] / df[annualized_volatility_col])
        df[vol_adj_trend_signal_col] = df[vol_adj_trend_signal_col].fillna(0)
        df[f'{ticker}_t_1_close'] = df[f'{ticker}'].shift(1)
        trend_cols = [f'{ticker}', f'{ticker}_t_1_close', f'{ticker}_pct_returns', trend_signal_col, trend_returns_col,
                      trend_trades_col, annualized_volatility_col, vol_adj_trend_signal_col]
        final_cols.append(trend_cols)
        ticker_signal_dict[ticker] = df[trend_cols]
    df_signal = pd.concat(ticker_signal_dict, axis=1)

    ## Assign new column names to the dataframe
    df_signal.columns = df_signal.columns.to_flat_index()
    final_cols = [item for sublist in final_cols for item in sublist]
    df_signal.columns = final_cols

    ## Normalize the weights of each position by the total weight of the portfolio
    vol_normalized_signal_cols = [f'{ticker}_vol_adjusted_trend_signal' for ticker in ticker_list]
    df_signal[vol_normalized_signal_cols] = df_signal[vol_normalized_signal_cols].fillna(0)
    for ticker in ticker_list:
        df_signal[f'{ticker}_position_volatility_adjusted_weight'] = (df_signal[f'{ticker}_vol_adjusted_trend_signal'] /
                                                                      df_signal[vol_normalized_signal_cols].abs()
                                                                      .sum(axis=1))
        df_signal[f'{ticker}_position_volatility_adjusted_weight'] = df_signal[
            f'{ticker}_position_volatility_adjusted_weight'].fillna(0)

    return df_signal

In [62]:
df_test = get_volatility_adjusted_trend_signal_timestamp(df_signal, ticker_list)

KeyError: 'BTC-USD'

In [64]:
df_signal.head()

,BTC-USD_8am_open_price,BTC-USD_8am_close_price,BTC-USD_pct_returns,BTC-USD_50_mavg,BTC-USD_90_mavg,BTC-USD_130_mavg,BTC-USD_170_mavg,BTC-USD_210_mavg,BTC-USD_250_mavg,BTC-USD_trend_signal,ETH-USD_8am_open_price,ETH-USD_8am_close_price,ETH-USD_pct_returns,ETH-USD_50_mavg,ETH-USD_90_mavg,ETH-USD_130_mavg,ETH-USD_170_mavg,ETH-USD_210_mavg,ETH-USD_250_mavg,ETH-USD_trend_signal,LTC-USD_8am_open_price,LTC-USD_8am_close_price,LTC-USD_pct_returns,LTC-USD_50_mavg,LTC-USD_90_mavg,LTC-USD_130_mavg,LTC-USD_170_mavg,LTC-USD_210_mavg,LTC-USD_250_mavg,LTC-USD_trend_signal,SOL-USD_8am_open_price,SOL-USD_8am_close_price,SOL-USD_pct_returns,SOL-USD_50_mavg,SOL-USD_90_mavg,SOL-USD_130_mavg,SOL-USD_170_mavg,SOL-USD_210_mavg,SOL-USD_250_mavg,SOL-USD_trend_signal,BTC-USD_20_donchian_upper_band_price,BTC-USD_20_donchian_lower_band_price,BTC-USD_20_donchian_middle_band_price,BTC-USD_20_donchian_signal,ETH-USD_20_donchian_upper_band_price,ETH-USD_20_donchian_lower_band_price,ETH-USD_20_donchian_middle_band_price,ETH-USD_20_donchian_signal,LTC-USD_20_donchian_upper_band_price,LTC-USD_20_donchian_lower_band_price,LTC-USD_20_donchian_middle_band_price,LTC-USD_20_donchian_signal,SOL-USD_20_donchian_upper_band_price,SOL-USD_20_donchian_lower_band_price,SOL-USD_20_donchian_middle_band_price,SOL-USD_20_donchian_signal,BTC-USD_50_6_250_mavg_crossover_20_donchian_signal,ETH-USD_50_6_250_mavg_crossover_20_donchian_signal,LTC-USD_50_6_250_mavg_crossover_20_donchian_signal,SOL-USD_50_6_250_mavg_crossover_20_donchian_signal,BTC-USD_volatility_20,BTC-USD_annualized_volatility_20,BTC-USD_vol_adjusted_trend_signal
2021-06-01,36601.20,36545.54,NaN,36601.200000,36601.200000,36601.200000,36601.200000,36601.200000,36601.200000,NaN,2624.78,2616.32,NaN,2624.780000,2624.780000,2624.780000,2624.780000,2624.780000,2624.780000,NaN,181.80,181.47,NaN,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0.0
2021-06-02,37204.26,37242.00,0.016477,36908.760600,36906.080333,36905.049462,36904.503706,36904.165857,36903.936120,-1.0,2695.44,2699.21,0.026920,2660.816600,2660.502556,2660.381769,2660.317824,2660.278238,2660.251320,-1.0,182.47,182.76,0.003685,182.141700,182.138722,182.137577,182.136971,182.136595,182.136340,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0.0
2021-06-03,38791.36,38811.27,0.042659,37561.559032,37548.523069,37543.517641,37540.869664,37539.231128,37538.117228,1.0,2808.70,2810.43,0.042019,2712.095726,2711.003483,2710.583721,2710.361574,2710.224083,2710.130602,1.0,192.28,192.21,0.053762,185.657194,185.594544,185.570529,185.557835,185.549982,185.544646,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0.0
2021-06-04,36587.77,36403.02,-0.056806,37303.314152,37300.269887,37299.038744,37298.373504,37297.957035,37297.671807,1.0,2625.76,2607.40,-0.065133,2689.199844,2688.977047,2688.885933,2688.836483,2688.805452,2688.784168,1.0,173.85,172.76,-0.095850,182.525974,182.559820,182.572435,182.579022,182.583069,182.585807,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0.0
2021-06-05,35915.26,35821.11,-0.018381,37003.059475,37010.821469,37013.702441,37015.203975,37016.125248,37016.748104,1.0,2642.08,2633.10,0.006215,2679.007191,2679.176196,2679.234512,2679.263870,2679.281516,2679.293285,1.0,173.75,173.04,-0.000575,180.627614,180.718686,180.753241,180.771427,180.782646,180.790258,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0.0


In [17]:
ticker_list = ['BTC-USD','ETH-USD','LTC-USD','SOL-USD']#,'XRP-USD']

In [64]:
start_quarter

datetime.date(2025, 1, 1)

In [152]:
start_date

'2025-01-01'

In [154]:
end_date

'2025-03-31'

In [158]:
ticker

'BTC-USD'

In [165]:
df_prices = get_coinbase_price_at_timestamp(client, ticker, start_date, end_date)

In [166]:
df_prices

,8am_open_price,8am_close_price
date,,
2025-01-01,93614.94,93583.67
2025-01-02,96754.55,96747.24
2025-01-03,96585.53,96591.16
2025-01-04,97853.76,97853.21
2025-01-05,97737.02,97737.01
2025-01-06,99131.85,99178.93
2025-01-07,100752.30,100727.87
2025-01-08,94852.71,94981.05
2025-01-09,93371.21,93354.87


In [ ]:
df

In [92]:
ticker_list = ['BTC-USD']#,'ETH-USD','LTC-USD','SOL-USD']#,'XRP-USD']
start_date = '2021-07-01'
end_date = '2025-02-24'
# start_date = '2025-02-01'
# end_date = '2025-02-24'
start_quarter = pd.Timestamp(start_date).to_period("Q").start_time.date()
end_quarter = pd.Timestamp(end_date).to_period("Q").end_time.date()
ticker_dict = {}
current_start = start_quarter#.strftime('%Y-%m-%d')
while current_start < end_quarter:
    current_end = pd.Timestamp(current_start).to_period("Q").end_time.date().strftime('%Y-%m-%d')
    for ticker in ticker_list:
        print(ticker, f'{current_start} to {current_end}')
        df = save_historical_crypto_prices_at_timestamp_from_coinbase(ticker=ticker, user_start_date=True, start_date=current_start.strftime('%Y-%m-%d'), end_date=current_end, save_to_file=False)
        ticker_dict[ticker] = df
        
    current_start = (pd.Timestamp(current_start) + pd.DateOffset(months=3)).to_period("Q").start_time.date()

BTC-USD 2025-01-01 to 2025-03-31


In [94]:
ticker_dict

{'BTC-USD':             8am_open_price  8am_close_price
 date                                       
 2025-01-01        93614.94         93583.67
 2025-01-02        96754.55         96747.24
 2025-01-03        96585.53         96591.16
 2025-01-04        97853.76         97853.21
 2025-01-05        97737.02         97737.01
 2025-01-06        99131.85         99178.93
 2025-01-07       100752.30        100727.87
 2025-01-08        94852.71         94981.05
 2025-01-09        93371.21         93354.87
 2025-01-10        95018.32         94997.79
 2025-01-11        94507.24         94544.88
 2025-01-12        94286.72         94298.56
 2025-01-13        90644.29         90509.77
 2025-01-14        95945.54         95999.32
 2025-01-15        96889.93         96891.48
 2025-01-16        99368.40         99284.16
 2025-01-17       102746.61        102879.91
 2025-01-18       103392.73        103405.45
 2025-01-19       104767.03        104823.45
 2025-01-20       106798.18        106696.79

In [100]:
print(current_start, current_end)

2025-01-01 2025-03-31


In [125]:
current_start = start_quarter#.strftime('%Y-%m-%d')
ticker_dict_test = {}
for ticker in ticker_list:
    print(ticker, f'{current_start} to {current_end}')
    df = save_historical_crypto_prices_at_timestamp_from_coinbase(ticker=ticker, user_start_date=True, start_date=current_start.strftime('%Y-%m-%d'), end_date=current_end, save_to_file=False)
    ticker_dict_test[ticker] = df

BTC-USD 2025-01-01 to 2025-03-31


In [133]:
df_test = save_historical_crypto_prices_at_timestamp_from_coinbase(ticker=ticker, user_start_date=True, start_date=current_start.strftime('%Y-%m-%d'), end_date=current_end, save_to_file=False)

In [129]:
current_end

'2025-03-31'

In [131]:
current_start

datetime.date(2025, 1, 1)

In [134]:
df_test

,8am_open_price,8am_close_price
date,,
2025-01-01,93614.94,93583.67
2025-01-02,96754.55,96747.24
2025-01-03,96585.53,96591.16
2025-01-04,97853.76,97853.21
2025-01-05,97737.02,97737.01
2025-01-06,99131.85,99178.93
2025-01-07,100752.30,100727.87
2025-01-08,94852.71,94981.05
2025-01-09,93371.21,93354.87


In [126]:
ticker_dict_test

{'BTC-USD':             8am_open_price  8am_close_price
 date                                       
 2025-01-01        93614.94         93583.67
 2025-01-02        96754.55         96747.24
 2025-01-03        96585.53         96591.16
 2025-01-04        97853.76         97853.21
 2025-01-05        97737.02         97737.01
 2025-01-06        99131.85         99178.93
 2025-01-07       100752.30        100727.87
 2025-01-08        94852.71         94981.05
 2025-01-09        93371.21         93354.87
 2025-01-10        95018.32         94997.79
 2025-01-11        94507.24         94544.88
 2025-01-12        94286.72         94298.56
 2025-01-13        90644.29         90509.77
 2025-01-14        95945.54         95999.32
 2025-01-15        96889.93         96891.48
 2025-01-16        99368.40         99284.16
 2025-01-17       102746.61        102879.91
 2025-01-18       103392.73        103405.45
 2025-01-19       104767.03        104823.45
 2025-01-20       106798.18        106696.79

In [111]:
current_start = start_quarter#.strftime('%Y-%m-%d')
ticker_dict = {}
# for ticker in ticker_list:
print(ticker, f'{current_start} to {current_end}')
df = save_historical_crypto_prices_at_timestamp_from_coinbase(ticker=ticker, user_start_date=True, start_date=current_start.strftime('%Y-%m-%d'), end_date=current_end, save_to_file=False)
ticker_dict[ticker] = df

BTC-USD 2025-01-01 to 2025-03-31


In [112]:
ticker_dict

{'BTC-USD':             8am_open_price  8am_close_price
 date                                       
 2025-01-01        93614.94         93583.67
 2025-01-02        96754.55         96747.24
 2025-01-03        96585.53         96591.16
 2025-01-04        97853.76         97853.21
 2025-01-05        97737.02         97737.01
 2025-01-06        99131.85         99178.93
 2025-01-07       100752.30        100727.87
 2025-01-08        94852.71         94981.05
 2025-01-09        93371.21         93354.87
 2025-01-10        95018.32         94997.79
 2025-01-11        94507.24         94544.88
 2025-01-12        94286.72         94298.56
 2025-01-13        90644.29         90509.77
 2025-01-14        95945.54         95999.32
 2025-01-15        96889.93         96891.48
 2025-01-16        99368.40         99284.16
 2025-01-17       102746.61        102879.91
 2025-01-18       103392.73        103405.45
 2025-01-19       104767.03        104823.45
 2025-01-20       106798.18        106696.79

In [103]:
ticker_dict

{'BTC-USD':             8am_open_price  8am_close_price
 date                                       
 2025-01-01        93614.94         93583.67
 2025-01-02        96754.55         96747.24
 2025-01-03        96585.53         96591.16
 2025-01-04        97853.76         97853.21
 2025-01-05        97737.02         97737.01
 2025-01-06        99131.85         99178.93
 2025-01-07       100752.30        100727.87
 2025-01-08        94852.71         94981.05
 2025-01-09        93371.21         93354.87
 2025-01-10        95018.32         94997.79
 2025-01-11        94507.24         94544.88
 2025-01-12        94286.72         94298.56
 2025-01-13        90644.29         90509.77
 2025-01-14        95945.54         95999.32
 2025-01-15        96889.93         96891.48
 2025-01-16        99368.40         99284.16
 2025-01-17       102746.61        102879.91
 2025-01-18       103392.73        103405.45
 2025-01-19       104767.03        104823.45
 2025-01-20       106798.18        106696.79

In [45]:
pd.Timestamp(start_quarter).to_period("Q").end_time.date().strftime('%Y-%m-%d')

'2021-09-30'

In [43]:
end_quarter.strftime('%Y-%m-%d')

'2025-03-31'

In [37]:
pd.Timestamp(start_date).to_period("Q").start_time.date()

datetime.date(2021, 7, 1)

In [35]:
pd.Timestamp(start_date).to_period("Q").end_time.date()

datetime.date(2021, 9, 30)

In [148]:
start_date = '2025-01-01'
end_date = '2025-03-31'
ticker_dict = {}
for ticker in ticker_list:
    print(ticker)
    df = save_historical_crypto_prices_at_timestamp_from_coinbase(ticker=ticker, user_start_date=True, start_date=start_date, end_date=end_date, save_to_file=False)
    ticker_dict[ticker] = df

BTC-USD


KeyError: "None of ['date'] are in the columns"

In [ ]:
df_prices_test = pd.concat(ticker_dict, axis=1)

In [ ]:
df_prices_test#.drop_duplicates()

In [20]:
df_prices = pd.concat(ticker_dict, axis=1)

In [21]:
df_prices

BTC-USD                        ETH-USD                  \
           8am_open_price 8am_close_price 8am_open_price 8am_close_price   
date                                                                       
2025-02-01      102258.15       102220.83        3266.59         3263.12   
2025-02-02       98141.85        98173.98        3051.09         3054.13   
2025-02-03       94850.00        94906.90        2583.18         2584.80   
2025-02-04       99460.67        99355.47        2807.38         2804.04   
2025-02-05       98103.77        98058.32        2788.26         2787.38   
2025-02-06       98374.78        98323.30        2771.61         2769.77   
2025-02-07       97998.69        97996.56        2758.30         2756.40   
2025-02-08       96060.27        96086.28        2604.18         2605.15   
2025-02-09       96563.48        96518.18        2655.48         2655.61   
2025-02-10       97390.38        97460.09        2639.93         2642.13   
2025-02-11       97792.18        97762.09        2686.74         2686.99   
2025-02-12       95977.19        95939.58        2628.34         2629.32   
2025-02-13       95837.52        95791.25        2655.94         2652.46   
2025-02-14       96660.59        96709.52        2686.58         2688.72   
2025-02-15       97758.04        97804.32        2710.58         2711.63   
2025-02-16       97140.73        97137.59        2706.00         2706.70   
2025-02-17       96459.15        96430.88        2766.03         2764.70   
2025-02-18       96154.80        96240.00        2702.59         2701.73   
2025-02-19       96139.68        96196.57        2704.47         2709.53   
2025-02-20       97437.40        97478.55        2742.27         2742.62   
2025-02-21       98931.26        99000.00        2807.21         2809.85   

                  LTC-USD                        SOL-USD                  
           8am_open_price 8am_close_price 8am_open_price 8am_close_price  
date                                                                      
2025-02-01         123.62          123.74         229.11          228.84  
2025-02-02         114.11          114.48         208.21          208.79  
2025-02-03          96.06           96.20         195.29          195.50  
2025-02-04         103.02          102.83         211.31          210.74  
2025-02-05         105.17          105.28         204.94          204.78  
2025-02-06         105.59          105.61         198.31          198.22  
2025-02-07         105.33          105.31         198.64          198.50  
2025-02-08         102.29          102.13         193.82          194.00  
2025-02-09         107.06          107.11         201.70          201.64  
2025-02-10         116.97          117.16         202.87          203.12  
2025-02-11         129.16          128.93         202.39          202.37  
2025-02-12         115.90          116.00         196.44          196.63  
2025-02-13         120.92          120.70         191.17          190.89  
2025-02-14         128.05          128.19         197.06          197.10  
2025-02-15         131.99          131.92         195.69          195.75  
2025-02-16         132.25          132.19         192.67          192.61  
2025-02-17         124.31          124.22         186.00          185.76  
2025-02-18         127.59          127.69         170.65          170.63  
2025-02-19         135.92          136.21         171.62          172.08  
2025-02-20         128.53          128.46         173.41          173.40  
2025-02-21         134.86          135.14         178.23          178.46

In [81]:
df = save_historical_crypto_prices_at_timestamp_from_coinbase(ticker=ticker, user_start_date=True, start_date='2025-02-01', end_date='2025-02-24', save_to_file=False)

In [83]:
df

,8am_open_price,8am_close_price
date,,
2025-02-01,229.11,228.84
2025-02-02,208.21,208.79
2025-02-03,195.29,195.50
2025-02-04,211.31,210.74
2025-02-05,204.94,204.78
2025-02-06,198.31,198.22
2025-02-07,198.64,198.50
2025-02-08,193.82,194.00
2025-02-09,201.70,201.64


In [ ]:
df_prices_test = get_coinbase_price_at_timestamp(client, ticker, start_date='2025-02-15', end_date='2025-02-21')

In [ ]:
df_prices_test

In [ ]:
df_prices_test

In [ ]:
df_prices_test

In [ ]:
start_date='2025-02-20'
end_date='2025-02-21'
date_list = pd.date_range(start=start_date, end=end_date, freq="D")
# for date in date_list:
date = date_list[0]
print(date)
start_timestamp = int(datetime.datetime.strptime(f'{date.date()} 08:00:00','%Y-%m-%d %H:%M:%S').timestamp())
end_timestamp = int(datetime.datetime.strptime(f'{date.date()} 08:00:00','%Y-%m-%d %H:%M:%S').timestamp())
candle_list = client.get_candles(
                    product_id=ticker,
                    start=start_timestamp,
                    end=end_timestamp,
                    granularity='ONE_MINUTE'  # Hourly candles
                ).candles

price_data = []
# Extract 8 AM open price
candles = candle_list[-1]  # [timestamp, low, high, open, close, volume]
price_data.append({
    'date': datetime.datetime.utcfromtimestamp(int(candles['start'])).date(),
    '8am_open_price': float(candles['open']),
    '8am_close_price': float(candles['close'])
})

In [ ]:
candle_list

In [ ]:
price_data

In [ ]:
price_data

In [ ]:
start_timestamp

In [ ]:
end_timestamp

In [ ]:
int(datetime.datetime.strptime(f'{"2025-02-20"} 07:59:00','%Y-%m-%d %H:%M:%S').timestamp())

In [ ]:
int(datetime.datetime.strptime(f'{"2025-02-20"} 08:01:10','%Y-%m-%d %H:%M:%S').timestamp())

In [ ]:
datetime.datetime.utcfromtimestamp(1740056400)

In [ ]:
candle_list

In [ ]:
candle_list[-1]

In [ ]:
candle_list

In [ ]:
candles

In [ ]:
%%time
df_prices = get_coinbase_8am_price(client, ticker)

In [ ]:
df_prices.head()

In [ ]:
start_timestamp

In [ ]:
df_market_trades = client.get_market_trades(product_id=ticker, limit=1000, start=1740056340, end=1740056470)

In [ ]:
df_market_trades['trades']

In [ ]:
df_candles, candle_list = get_coinbase_daily_historical_price_data_hour(client, ticker, start_timestamp, end_timestamp)

In [ ]:
df_candles

In [ ]:
candle_list

In [ ]:
for candles in candle_list:
    print(datetime.datetime.utcfromtimestamp(int(candles['start'])), candles['low'])